# DRE ATUARIAL

## PACOTES

In [1]:
import pandas as pd
from datetime import datetime,date
from pandasql import sqldf
from life.insurance import isrc_discrete

## Criando Vetores de Anuidades

In [2]:
lt = pd.read_csv('life/life_tables.csv')

In [4]:
lt.tail()

,AGE,AT_2000_F,AT_2000_M,AT_49_F,AT_49_M,AT_83_F,AT_83_M,IBGE_2020_F,IBGE_2020_M
111,111,0.644427,0.668186,1.0,1.0,0.650646,0.695704,1.0,1.0
112,112,0.719484,0.739483,1.0,1.0,0.724750,0.762343,1.0,1.0
113,113,0.803380,0.818254,1.0,1.0,0.807316,0.835056,1.0,1.0
114,114,0.896693,0.904945,1.0,1.0,0.898885,0.914167,1.0,1.0
115,115,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0


In [5]:
lt.head()

,AGE,AT_2000_F,AT_2000_M,AT_49_F,AT_49_M,AT_83_F,AT_83_M,IBGE_2020_F,IBGE_2020_M
0,0,0.001794,0.002311,0.003210,0.004040,0.001835,0.002690,0.010635,0.012426
1,1,0.000755,0.000906,0.001360,0.001580,0.000778,0.001053,0.000710,0.000861
2,2,0.000392,0.000504,0.000703,0.000887,0.000402,0.000591,0.000446,0.000570
3,3,0.000290,0.000408,0.000521,0.000715,0.000298,0.000476,0.000334,0.000441
4,4,0.000232,0.000357,0.000419,0.000627,0.000240,0.000417,0.000270,0.000367


In [3]:
from life.annuity import ann_discrete 
ax_discrete_f_v = ann_discrete(table=lt['IBGE_2020_F'])
ax_discrete_m_v = ann_discrete(table=lt['IBGE_2020_M'])
ax_discrete_f_inv = ann_discrete(table=lt['AT_2000_F'])
ax_discrete_m_inv = ann_discrete(table=lt['AT_2000_M'])

In [4]:
idades = lt['AGE']
axs= pd.DataFrame(columns=['IDADE','AX_FEM_VALIDOS','AX_MASC_VALIDOS','AX_FEM_INVALIDOS','AX_MASC_INVALIDOS'])

In [5]:
for i in range(len(idades)):
    ax_f_v = ax_discrete_f_v.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_m_v = ax_discrete_m_v.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_m_inv = ax_discrete_f_inv.ax(i=0.05, age=idades[i], B=1, due=True)
    ax_f_inv = ax_discrete_m_inv.ax(i=0.05, age=idades[i], B=1, due=True)
    axs = pd.DataFrame([[idades[i], ax_f_v,ax_m_v,ax_m_inv,ax_f_inv]],columns = axs.columns).append(axs)


In [6]:
axs = axs.sort_values('IDADE')

In [7]:
axs.reset_index(inplace = True)

In [8]:
axs.drop(columns='index',inplace=True)

In [9]:
axs.head()

,IDADE,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS
0,0,20.194011,19.828379,20.485233,20.317675
1,1,20.370353,20.018549,20.496265,20.330542
2,2,20.353317,19.986675,20.486546,20.315475
3,3,20.330045,19.947385,20.468897,20.291476
4,4,20.303319,19.903541,20.448272,20.264317


## Criando Vetor de lx

In [10]:
pop_inic = 100000
idade_inic = 0

In [11]:
d = {
    'IDADE' : [idade_inic] ,
    'LX_FEM_VALIDOS' : [pop_inic],
    'LX_MASC_VALIDOS' :[pop_inic],
    'LX_FEM_INVALIDOS' : [pop_inic],
    'LX_MASC_INVALIDOS' : [pop_inic]
}

In [12]:
lxs= pd.DataFrame(data=d)
c = 0

In [13]:
for i in range(len(axs)):
    if i == 0:
        lxs['LX_FEM_VALIDOS'][i] = pop_inic
        lxs['LX_MASC_VALIDOS'][i] = pop_inic
        lxs['LX_FEM_INVALIDOS'][i] = pop_inic
        lxs['LX_MASC_INVALIDOS'][i] = pop_inic
    else:
        lx_f_v = (1-lt['IBGE_2020_F'][i-1]) * lxs['LX_FEM_VALIDOS'][i-1]
        lx_m_v = (1-lt['IBGE_2020_M'][i-1]) * lxs['LX_MASC_VALIDOS'][i-1]
        lx_m_inv = (1-lt['AT_2000_F'][i-1]) * lxs['LX_FEM_INVALIDOS'][i-1]
        lx_f_inv = (1-lt['AT_2000_M'][i-1]) * lxs['LX_MASC_INVALIDOS'][i-1]
        lxs = lxs.append({'IDADE' : c ,
                            'LX_FEM_VALIDOS' :  lx_f_v,
                            'LX_MASC_VALIDOS' : lx_m_v,
                            'LX_FEM_INVALIDOS' : lx_m_inv,
                            'LX_MASC_INVALIDOS' :  lx_f_inv},ignore_index=True)
    c = c + 1

In [14]:
lxs['IDADE'] = lxs['IDADE'].astype(int)

In [15]:
lxs.head()

,IDADE,LX_FEM_VALIDOS,LX_MASC_VALIDOS,LX_FEM_INVALIDOS,LX_MASC_INVALIDOS
0,0,100000.000000,100000.000000,100000.000000,100000.000000
1,1,98936.486599,98757.397678,99820.600000,99768.900000
2,2,98866.259247,98672.413369,99745.235447,99678.509377
3,3,98822.191845,98616.143082,99706.135315,99628.271408
4,4,98789.230571,98572.607393,99677.220535,99587.623073


## APOSENTADOS

In [17]:
aposentados = pd.read_csv('Planilhas-Tratadas/APOSENTADOS.csv',delimiter=";")

In [3]:
aposentados.head()

,Unnamed: 0,NU_ANO,NU_MES,CO_IBGE,NO_ENTE,SG_UF,CO_COMP_MASSA,CO_TIPO_FUNDO,CNPJ_ORGAO,NO_ORGAO,...,VL_CONTRIBUICAO,VL_COMPENS_PREVID,IN_PARID_SERV,CO_CONDICAO_APOSENTADO,NU_DEPENDENTES,NU_TEMPO_RPPS_MUN,NU_TEMPO_RPPS_EST,NU_TEMPO_RPPS_FED,IN_PREV_COMP,VL_TETO_ESPECIFICO
0,0,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
1,1,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
2,2,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,2,1,1,0,0,0,2,29000
3,3,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000
4,4,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0.0,0,1,1,1,0,0,0,2,29000


In [4]:
aposentados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6782 entries, 0 to 6781
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               6782 non-null   int64  
 1   NU_ANO                   6782 non-null   int64  
 2   NU_MES                   6782 non-null   int64  
 3   CO_IBGE                  6782 non-null   int64  
 4   NO_ENTE                  6782 non-null   object 
 5   SG_UF                    6782 non-null   object 
 6   CO_COMP_MASSA            6782 non-null   int64  
 7   CO_TIPO_FUNDO            6782 non-null   int64  
 8   CNPJ_ORGAO               6782 non-null   int64  
 9   NO_ORGAO                 6782 non-null   object 
 10  CO_PODER                 6782 non-null   int64  
 11  CO_TIPO_PODER            6782 non-null   int64  
 12  CO_TIPO_POPULACAO        6782 non-null   int64  
 13  CO_TIPO_CARGO            6782 non-null   int64  
 14  CO_TIPO_APOSENTADORIA   

In [16]:
aposentados['DT_NASC_APOSENTADO'] = pd.to_datetime(aposentados['DT_NASC_APOSENTADO'])

NameError: name 'aposentados' is not defined

In [6]:
today_date =  datetime.now()
today_date = today_date.strftime( "%d-%m-%Y")
today_date = pd.to_datetime(today_date)

In [7]:
def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [8]:
aposentados['IDADE'] = aposentados['DT_NASC_APOSENTADO'].apply(age)

### (-) Valor Presente dos Benefícios Futuros (Aposentados)

In [15]:
aposentados.columns

Index(['Unnamed: 0', 'NU_ANO', 'NU_MES', 'CO_IBGE', 'NO_ENTE', 'SG_UF',
       'CO_COMP_MASSA', 'CO_TIPO_FUNDO', 'CNPJ_ORGAO', 'NO_ORGAO', 'CO_PODER',
       'CO_TIPO_PODER', 'CO_TIPO_POPULACAO', 'CO_TIPO_CARGO',
       'CO_TIPO_APOSENTADORIA', 'ID_APOSENTADO_MATRICULA', 'ID_APOSENTADO_CPF',
       'ID_APOSENT_PIS_PASEP', 'CO_SEXO_APOSENTADO', 'CO_EST_CIVIL_APOSENTADO',
       'DT_NASC_APOSENTADO', 'DT_ING_SERV_PUB', 'DT_ING_ENTE',
       'DT_INICIO_APOSENTADORIA', 'VL_APOSENTADORIA', 'VL_CONTRIBUICAO',
       'VL_COMPENS_PREVID', 'IN_PARID_SERV', 'CO_CONDICAO_APOSENTADO',
       'NU_DEPENDENTES', 'NU_TEMPO_RPPS_MUN', 'NU_TEMPO_RPPS_EST',
       'NU_TEMPO_RPPS_FED', 'IN_PREV_COMP', 'VL_TETO_ESPECIFICO', 'IDADE'],
      dtype='object')

In [16]:
vpbf_aposent = pd.merge(aposentados[['IDADE','CO_SEXO_APOSENTADO','CO_CONDICAO_APOSENTADO','VL_APOSENTADORIA']],
                        axs[['IDADE','AX_FEM_VALIDOS','AX_MASC_VALIDOS','AX_FEM_INVALIDOS','AX_MASC_INVALIDOS']],
                        on='IDADE',
                        how='inner')

In [17]:
vpbf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015


In [18]:
p = 13
vpbf_aposent['VALOR_BENF_FUT'] = 0

In [19]:
for i in range(len(vpbf_aposent)):
   if vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p
      
   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_INVALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

   elif vpbf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpbf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_INVALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p

<ipython-input-19-b50e1364238a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_MASC_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p
<ipython-input-19-b50e1364238a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_aposent['VALOR_BENF_FUT'][i] = vpbf_aposent['AX_FEM_VALIDOS'][i] * vpbf_aposent['VL_APOSENTADORIA'][i] * p


In [20]:
vpbf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS,VALOR_BENF_FUT
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015,1315808
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015,1216523
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015,1591067
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015,691387
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015,1404525


In [21]:
vpbf_aposent['VALOR_BENF_FUT'].sum()

10055467524

### (+) Valor Presente das Contribuições Futuras Participante (Aposentados)

In [22]:
vpcf_aposent = vpbf_aposent.copy()

In [23]:
vpcf_aposent.head()

,IDADE,CO_SEXO_APOSENTADO,CO_CONDICAO_APOSENTADO,VL_APOSENTADORIA,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS,VALOR_BENF_FUT
0,65,2,1,8972.34,12.595562,11.280892,13.336076,12.278015,1315808
1,65,2,1,8295.33,12.595562,11.280892,13.336076,12.278015,1216523
2,65,1,1,9716.90,12.595562,11.280892,13.336076,12.278015,1591067
3,65,1,1,4222.41,12.595562,11.280892,13.336076,12.278015,691387
4,65,1,1,8577.66,12.595562,11.280892,13.336076,12.278015,1404525


In [24]:
vpcf_aposent['VALOR_CONTRIB_FUT'] = 0
t = 0.14
teto = 7087.22
p = 1

In [25]:
for i in range(len(vpbf_aposent)):
  if vpcf_aposent['VL_APOSENTADORIA'][i] > teto:
    if vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
      
    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 1:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t

    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 1 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_INVALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t

    elif vpcf_aposent['CO_SEXO_APOSENTADO'][i] == 2 and vpcf_aposent['CO_CONDICAO_APOSENTADO'][i] == 2:
      vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_INVALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t 
  else:
    vpcf_aposent['VALOR_CONTRIB_FUT'][i] = 0

<ipython-input-25-6f8ecb3f1884>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_MASC_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
<ipython-input-25-6f8ecb3f1884>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpcf_aposent['VALOR_CONTRIB_FUT'][i] = p * (vpcf_aposent['AX_FEM_VALIDOS'][i] * vpcf_aposent['VL_APOSENTADORIA'][i]) * ((vpcf_aposent['VL_APOSENTADORIA'][i] - teto ) / vpcf_aposent['VL_APOSENTADORIA'][i]) * t
<ipython-input-25-6f8ecb3f1884>:15: SettingWi

In [26]:
vpcf_aposent['VALOR_CONTRIB_FUT'].sum()

33378465

## PENSIONISTAS

In [16]:
pensionistas = pd.read_csv('Planilhas-Tratadas/PENSIONISTAS.csv',delimiter=";")

In [17]:
pensionistas['DT_NASC_PENSIONISTA'] = pd.to_datetime(pensionistas['DT_NASC_PENSIONISTA'])

In [18]:
today_date =  datetime.now()
today_date = today_date.strftime( "%d-%m-%Y")
today_date = pd.to_datetime(today_date)

In [19]:
def age(born):
    today = date.today()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [20]:
pensionistas['IDADE'] = pensionistas['DT_NASC_PENSIONISTA'].apply(age)

In [21]:
pensionistas['IDADE_TEMP'] = pensionistas['IDADE'] + pensionistas['NU_TEMPO_DURACAO'] 

In [22]:
pensionistas.head()

,Unnamed: 0,NU_ANO,NU_MES,CO_IBGE,NO_ENTE,SG_UF,CO_COMP_MASSA,CO_TIPO_FUNDO,CNPJ_ORGAO,NO_ORGAO,...,VL_CONTRIBUICAO,VL_COMPENS_PREVID,IN_PARID_SERV,CO_CONDICAO,CO_DURACAO,NU_TEMPO_DURACAO,IN_PREV_COMP,VL_TETO_ESPECIFICO,IDADE,IDADE_TEMP
0,0,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0,0,2,1,2,13,2,29000,9,22
1,1,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0,0,2,1,1,0,2,29000,44,44
2,2,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0,0,2,1,1,0,2,29000,46,46
3,3,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0,0,2,1,2,15,2,29000,8,23
4,4,2021,12,2306504,Itapiúna,Ceará,1,1,3333333,Instituto,...,0,0,2,1,1,0,2,29000,32,32


### (-) Valor Presente dos Benefícios Futuros (Pensionistas)

In [23]:
vpbf_pens = pensionistas[['IDADE','IDADE_TEMP','CO_SEXO_PENSIONISTA','CO_CONDICAO','CO_DURACAO','NU_TEMPO_DURACAO','VL_BENEF_PENSAO']]

In [24]:
vpbf_pens.head()

,IDADE,IDADE_TEMP,CO_SEXO_PENSIONISTA,CO_CONDICAO,CO_DURACAO,NU_TEMPO_DURACAO,VL_BENEF_PENSAO
0,9,22,1,1,2,13,4717.59
1,44,44,1,1,1,0,4976.62
2,46,46,2,1,1,0,3043.72
3,8,23,1,1,2,15,3791.64
4,32,32,1,1,1,0,4157.90


In [25]:
axs.head()

,IDADE,AX_FEM_VALIDOS,AX_MASC_VALIDOS,AX_FEM_INVALIDOS,AX_MASC_INVALIDOS
0,0,20.194011,19.828379,20.485233,20.317675
1,1,20.370353,20.018549,20.496265,20.330542
2,2,20.353317,19.986675,20.486546,20.315475
3,3,20.330045,19.947385,20.468897,20.291476
4,4,20.303319,19.903541,20.448272,20.264317


In [26]:
lxs.head()

,IDADE,LX_FEM_VALIDOS,LX_MASC_VALIDOS,LX_FEM_INVALIDOS,LX_MASC_INVALIDOS
0,0,100000.000000,100000.000000,100000.000000,100000.000000
1,1,98936.486599,98757.397678,99820.600000,99768.900000
2,2,98866.259247,98672.413369,99745.235447,99678.509377
3,3,98822.191845,98616.143082,99706.135315,99628.271408
4,4,98789.230571,98572.607393,99677.220535,99587.623073


In [55]:
vpbf_pens['VALOR_BENF_FUT'] = 0
p = 13
i = 0.14
v = (1/(1+i))

<ipython-input-55-9f2cfbb98bb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_pens['VALOR_BENF_FUT'] = 0


In [56]:
for i in range(len(vpbf_pens)):

   if  vpbf_pens['CO_DURACAO'][i] == 1: # cod vitalicio

      if vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpbf_pens['CO_CONDICAO'][i] == 1:

         ax_fem_validos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_FEM_VALIDOS']]['AX_FEM_VALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = (p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_fem_validos) 

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpbf_pens['CO_CONDICAO'][i] == 1:

         ax_masc_validos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_MASC_VALIDOS']]['AX_MASC_VALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = (p * vpbf_pens['VL_BENEF_PENSAO'][i] * ax_masc_validos) 

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpbf_pens['CO_CONDICAO'][i] == 2:

         ax_fem_invalidos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_FEM_INVALIDOS']]['AX_FEM_INVALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = (p * vpbf_pens['VL_BENEF_PENSAO'][i] * ax_fem_invalidos)

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpbf_pens['CO_CONDICAO'][i] == 2:

         ax_masc_invalidos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_MASC_INVALIDOS']]['AX_MASC_INVALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = (p * vpbf_pens['VL_BENEF_PENSAO'][i] * ax_masc_invalidos)

   elif  vpbf_pens['CO_DURACAO'][i] == 2: # cod pensionista

      if vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpbf_pens['CO_CONDICAO'][i] == 1:

         ax_fem_validos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_FEM_VALIDOS']]['AX_FEM_VALIDOS'].iloc[0]
         ax_fem_validos_nu = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['AX_FEM_VALIDOS']]['AX_FEM_VALIDOS'].iloc[0]
         lx_fem_validos = lxs.loc[(lxs['IDADE'] == vpbf_pens['IDADE'][i]),['LX_FEM_VALIDOS']]['LX_FEM_VALIDOS'].iloc[0]
         lx_fem_validos_nu = lxs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['LX_FEM_VALIDOS']]['LX_FEM_VALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = ( p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_fem_validos) - ((v ** vpbf_pens['NU_TEMPO_DURACAO'][i]) * (lx_fem_validos_nu/lx_fem_validos) *  ax_fem_validos_nu)

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpbf_pens['CO_CONDICAO'][i] == 1:

         ax_masc_validos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_MASC_VALIDOS']]['AX_MASC_VALIDOS'].iloc[0]
         ax_masc_validos_nu = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['AX_MASC_VALIDOS']]['AX_MASC_VALIDOS'].iloc[0]
         lx_masc_validos = lxs.loc[(lxs['IDADE'] == vpbf_pens['IDADE'][i]),['LX_MASC_VALIDOS']]['LX_MASC_VALIDOS'].iloc[0]
         lx_masc_validos_nu = lxs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['LX_MASC_VALIDOS']]['LX_MASC_VALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = ( p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_masc_validos) - ((v ** vpbf_pens['NU_TEMPO_DURACAO'][i]) * (lx_masc_validos_nu/lx_masc_validos) *  ax_masc_validos_nu)

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpbf_pens['CO_CONDICAO'][i] == 2:

         ax_fem_invalidos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_FEM_INVALIDOS']]['AX_FEM_INVALIDOS'].iloc[0]
         ax_fem_invalidos_nu = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['AX_FEM_INVALIDOS']]['AX_FEM_INVALIDOS'].iloc[0]
         lx_fem_invalidos = lxs.loc[(lxs['IDADE'] == vpbf_pens['IDADE'][i]),['LX_FEM_INVALIDOS']]['LX_FEM_INVALIDOS'].iloc[0]
         lx_fem_invalidos_nu = lxs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['LX_FEM_INVALIDOS']]['LX_FEM_INVALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = ( p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_fem_invalidos) - ((v ** vpbf_pens['NU_TEMPO_DURACAO'][i]) * (lx_fem_invalidos_nu/lx_fem_invalidos) *  ax_fem_invalidos_nu)

      elif vpbf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpbf_pens['CO_CONDICAO'][i] == 2:
         
         ax_masc_invalidos = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE'][i]),['AX_MASC_INVALIDOS']]['AX_MASC_INVALIDOS'].iloc[0]
         ax_masc_invalidos_nu = axs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['AX_MASC_INVALIDOS']]['AX_MASC_INVALIDOS'].iloc[0]
         lx_masc_invalidos = lxs.loc[(lxs['IDADE'] == vpbf_pens['IDADE'][i]),['LX_MASC_INVALIDOS']]['LX_MASC_INVALIDOS'].iloc[0]
         lx_masc_invalidos_nu = lxs.loc[(axs['IDADE'] == vpbf_pens['IDADE_TEMP'][i]),['LX_MASC_INVALIDOS']]['LX_MASC_INVALIDOS'].iloc[0]
         vpbf_pens['VALOR_BENF_FUT'][i] = ( p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_masc_invalidos) - ((v ** vpbf_pens['NU_TEMPO_DURACAO'][i]) * (lx_masc_invalidos_nu/lx_masc_invalidos) *  ax_masc_invalidos_nu)

<ipython-input-56-28293a495d35>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpbf_pens['VALOR_BENF_FUT'][i] = ( p * vpbf_pens['VL_BENEF_PENSAO'][i] *  ax_fem_validos) - ((v ** vpbf_pens['NU_TEMPO_DURACAO'][i]) * (lx_fem_validos_nu/lx_fem_validos) *  ax_fem_validos_nu)
/home/alex/.virtualenvs/etl/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3398: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-56-28293a495d35>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [57]:
vpbf_pens['VALOR_BENF_FUT'].sum()

306718413

## (+) Valor Presente das Contribuições Futuras Participante (Pensionistas)

In [58]:
vpcf_pens = vpbf_pens.copy()

In [59]:
vpcf_pens.head()

,IDADE,IDADE_TEMP,CO_SEXO_PENSIONISTA,CO_CONDICAO,CO_DURACAO,NU_TEMPO_DURACAO,VL_BENEF_PENSAO,VALOR_BENF_FUT
0,9,22,1,1,2,13,4717.59,1234829
1,44,44,1,1,1,0,4976.62,1108418
2,46,46,2,1,1,0,3043.72,622178
3,8,23,1,1,2,15,3791.64,994331
4,32,32,1,1,1,0,4157.90,1008467


In [60]:
vpcf_pens['VALOR_CONTRIB_FUT'] = 0
t = 0.14
teto = 7087.22
p = 13

In [61]:
for i in range(len(vpcf_pens)):
  if vpcf_pens['VL_BENEF_PENSAO'][i] > teto:

    if vpcf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpcf_pens['CO_CONDICAO'][i] == 1:

      ax_fem_validos = axs.loc[(axs['IDADE'] == vpcf_pens['IDADE'][i]),['AX_FEM_VALIDOS']]['AX_FEM_VALIDOS'].iloc[0]
      vpcf_pens['VALOR_CONTRIB_FUT'][i] = p * (ax_fem_validos * vpcf_pens['VL_BENEF_PENSAO'][i]) * ((vpcf_pens['VL_BENEF_PENSAO'][i] - teto ) / vpcf_pens['VL_BENEF_PENSAO'][i]) * t
      
    elif vpcf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpcf_pens['CO_CONDICAO'][i] == 1:

      ax_masc_validos = axs.loc[(axs['IDADE'] == vpcf_pens['IDADE'][i]),['AX_MASC_VALIDOS']]['AX_MASC_VALIDOS'].iloc[0]
      vpcf_pens['VALOR_CONTRIB_FUT'][i] = p * (ax_masc_validos * vpcf_pens['VL_BENEF_PENSAO'][i]) * ((vpcf_pens['VL_BENEF_PENSAO'][i] - teto ) / vpcf_pens['VL_BENEF_PENSAO'][i]) * t

    elif vpcf_pens['CO_SEXO_PENSIONISTA'][i] == 1 and vpcf_pens['CO_CONDICAO'][i] == 2:

      ax_fem_invalidos = axs.loc[(axs['IDADE'] == vpcf_pens['IDADE'][i]),['AX_FEM_INVALIDOS']]['AX_FEM_INVALIDOS'].iloc[0]
      vpcf_pens['VALOR_CONTRIB_FUT'][i] = p * (ax_fem_invalidos * vpcf_pens['VL_BENEF_PENSAO'][i]) * ((vpcf_pens['VL_BENEF_PENSAO'][i] - teto ) / vpcf_pens['VL_BENEF_PENSAO'][i]) * t

    elif vpcf_pens['CO_SEXO_PENSIONISTA'][i] == 2 and vpcf_pens['CO_CONDICAO'][i] == 2:

      ax_masc_invalidos = axs.loc[(axs['IDADE'] == vpcf_pens['IDADE'][i]),['AX_MASC_INVALIDOS']]['AX_MASC_INVALIDOS'].iloc[0]
      vpcf_pens['VALOR_CONTRIB_FUT'][i] = p * (ax_masc_invalidos * vpcf_pens['VL_BENEF_PENSAO'][i]) * ((vpcf_pens['VL_BENEF_PENSAO'][i] - teto ) / vpcf_pens['VL_BENEF_PENSAO'][i]) * t 
  
  else:

    vpcf_pens['VALOR_CONTRIB_FUT'][i] = 0

<ipython-input-61-9e9bce227e79>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vpcf_pens['VALOR_CONTRIB_FUT'][i] = 0


In [65]:
vpcf_pens['VALOR_CONTRIB_FUT'].sum()

0